# Milestone 3: Graph-RAG Implementation

**Theme:** Hotel

**Task:** Hybrid (Hotel Recommendation + Visa Assistant)

**Retrieval Approach:** Knowledge Graph + Embeddings (Graph-RAG)

---

## Table of Contents

1. [Part 1: Input Preprocessing](#part1)
   - [1.a Intent Classification](#part1a)
   - [1.b Entity Extraction](#part1b)
   - [1.c Input Embedding](#part1c)
2. [Part 2: Graph Retrieval + Experiments](#part2)
   - [2.a Baseline (Cypher Queries)](#part2a)
   - [2.b Embeddings-Based Retrieval (2 model comparison)](#part2b)
3. [Part 3: LLM Layer + Experiments](#part3)
   - [3.a Context Construction](#part3a)
   - [3.b Prompt Engineering](#part3b)
   - [3.c LLM Comparison (3 models)](#part3c)
4. [Part 4: UI + Full Pipeline Demo](#part4)
   - [4.a Streamlit Interface](#part4a)
   - [4.b End-to-End Demonstration](#part4b)

---
<a id='part1'></a>
# Part 1: Input Preprocessing

This section preprocesses natural language user queries into structured data for graph retrieval.

**Three components:**

**1.a Intent Classification** - Determines user's goal using action-based intents (LIST_HOTELS, RECOMMEND_HOTEL, DESCRIBE_HOTEL, COMPARE_HOTELS, CHECK_VISA)

**1.b Entity Extraction** - Extracts specific information (hotel names, cities, countries, traveler demographics, preferences)

**1.c Input Embedding** - Converts user query into vector representation for semantic similarity search (Part 2.b)

<a id='part1a'></a>
## 1.a Intent Classification

### Purpose
Classify user queries into specific intent categories to determine:
- What type of information the user needs
- Which Cypher queries to execute
- How to structure the response

### Design Decision: LLM-Based Classification

We use an **LLM-based approach** (OpenAI GPT-4o-mini) because:
- Handles natural language variations effectively
- Understands context and nuance
- Can apply complex tie-breaking rules
- Flexible for conversational queries

### Defined Intents

We define **5 action-based intents** that represent user goals:

| Intent | Purpose | Keywords | Example Queries |
|--------|---------|----------|----------------|
| **`LIST_HOTELS`** | Find multiple hotels matching filters | "show", "find", "list" | - "Show me hotels in Paris"<br>- "Find 5-star hotels in Dubai" |
| **`RECOMMEND_HOTEL`** | Get personalized suggestions or best options | "recommend", "suggest", "best", "top" | - "Recommend a hotel for families in Cairo"<br>- "What's the best cheap hotel in Rome?" |
| **`DESCRIBE_HOTEL`** | Get detailed info about a specific hotel | hotel name mentioned | - "Tell me about Hilton Cairo"<br>- "What is The Azure Tower like?" |
| **`COMPARE_HOTELS`** | Compare two or more hotels | "compare", "vs", "which is better" | - "Compare Hilton and Marriott in Cairo"<br>- "Which is better: Hotel A or Hotel B?" |
| **`CHECK_VISA`** | Visa requirements between countries | "visa", "entry requirement" | - "Do Egyptians need a visa for Turkey?"<br>- "Visa rules from UK to UAE" |

---

### Intent Design Philosophy

- **Action-based naming** — Each intent name describes what the user wants to DO
- **Clear semantic boundaries** — LIST (neutral) vs RECOMMEND (opinion) vs DESCRIBE (specific)
- **Explicit tie-breaking rules** — Clear keyword priorities reduce confusion

---

### Out-of-Scope Handling

If the query does not match any of the 5 intents  
(e.g., *"What's the weather?"*, *"Tell me a joke"*),  
the classifier returns `None`, and the system responds:

> **"I cannot help with this request. I can assist with hotel search, recommendations, visa information, and hotel comparisons."**

In [1]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from typing import Dict, List, Any, Optional
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

print("Libraries imported successfully")

Libraries imported successfully


### Implementation: LLM-Based Intent Classifier with OpenAI

The classifier uses a structured prompt with:
- Clear intent definitions
- Positive and negative examples for each intent
- Explicit tie-breaking rules to reduce ambiguity

In [3]:
class IntentClassifier:
    
    def __init__(self):
        # Load OpenAI API key
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY not found in environment variables")
        
        # Initialize OpenAI client
        self.client = OpenAI(api_key=self.api_key)
        
        # Model configuration
        self.model = "gpt-4o-mini"  # Fast and cost-effective for classification
        
        # Define intents with descriptions
        self.intents = {
            "LIST_HOTELS": "Find multiple hotels matching filters (neutral listing)",
            "RECOMMEND_HOTEL": "Get personalized suggestions or best options (opinion/advice)",
            "DESCRIBE_HOTEL": "Get detailed information about one specific hotel",
            "COMPARE_HOTELS": "Compare two or more specific hotels",
            "CHECK_VISA": "Check visa requirements between countries"
        }
    
    def classify(self, user_query: str) -> Optional[str]:
        """
        Classify user query into one of 5 intents using LLM.
        Returns intent name or None if out-of-scope.
        """
        prompt = self._build_prompt(user_query)
        
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a precise intent classifier for a hotel search system. Return ONLY the intent name, nothing else."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.0,
                max_tokens=20
            )
            
            intent = response.choices[0].message.content.strip().upper()
            
            # Validate response
            if intent in self.intents.keys():
                return intent
            elif intent == "NONE":
                return None
            else:
                print(f"Warning: OpenAI returned unexpected value '{intent}', treating as out-of-scope")
                return None
                
        except Exception as e:
            print(f"Error calling OpenAI API: {e}")
            return None
    
    def _build_prompt(self, user_query: str) -> str:
        """Build the classification prompt with clear examples and tie-breaking rules."""
        
        return f"""Classify the following user query into ONE of these intents, or return "NONE" if it doesn't match any:

         ═══════════════════════════════════════════════════════════════════════════════

         **1. LIST_HOTELS**
            Purpose: User wants a neutral list of hotels matching filters
            Keywords: "show", "find", "list", "search for", "hotels in"
            
            Examples:
            - "Show me hotels in Paris"
            - "Find hotels in Dubai"
            - "List 5-star hotels in Rome"
            - "Hotels near the city center in Cairo"
            - "Search for hotels in Berlin"

         ═══════════════════════════════════════════════════════════════════════════════

         **2. RECOMMEND_HOTEL**
            Purpose: User wants advice, suggestions, or "best" options
            Keywords: "recommend", "suggest", "best", "top", "which should I", "what do you recommend"
            Can include personal preferences or demographics
            
            Examples:
            - "Recommend a hotel for families in Dubai"
            - "What's the best cheap hotel in Paris?"
            - "I'm 25 traveling solo, suggest a hotel in Cairo"
            - "Top hotels in Rome"
            - "Which hotel should I book in Berlin?"
            - "Suggest a romantic hotel for couples"

         ═══════════════════════════════════════════════════════════════════════════════

         **3. DESCRIBE_HOTEL**
            Purpose: Get detailed information about ONE specific hotel
            Keywords: "tell me about", "what is", "information on", "describe"
            Must mention a specific hotel name
            
            Examples:
            - "Tell me about Hilton Cairo"
            - "What is The Azure Tower like?"
            - "Information about Marriott Dubai"
            - "Describe the Grand Hotel Paris"
            - "What can you tell me about Hotel Adlon?"

         ═══════════════════════════════════════════════════════════════════════════════

         **4. COMPARE_HOTELS**
            Purpose: Compare TWO or MORE specific hotels
            Keywords: "compare", "vs", "versus", "which is better", "difference between"
            Must mention multiple hotels
            
            Examples:
            - "Compare Hilton Cairo and Marriott Cairo"
            - "Which is better: The Azure Tower or Nile Grandeur?"
            - "Hotel A vs Hotel B in Paris"
            - "Difference between Sheraton and Radisson in Dubai"
            - "Compare these three hotels: X, Y, and Z"

         ═══════════════════════════════════════════════════════════════════════════════

         **5. CHECK_VISA**
            Purpose: Check visa requirements between countries
            Keywords: "visa", "entry requirement", "travel document", "do I need"
            
            Examples:
            - "Do Egyptians need a visa for Turkey?"
            - "Visa requirements from UK to UAE"
            - "Do Indians need a visa to visit Germany?"
            - "What are the entry requirements for France?"
            - "Travel document requirements from US to Japan"

         ═══════════════════════════════════════════════════════════════════════════════

         **TIE-BREAKING RULES (VERY IMPORTANT):**

         1. If the query contains ANY of: "recommend", "suggest", "best", "top", "which should I"
            -> ALWAYS choose RECOMMEND_HOTEL (never LIST_HOTELS)

         2. If the query mentions personal info (age, traveler type: "family", "solo", "couple", "business")
            -> Choose RECOMMEND_HOTEL

         3. If the query mentions TWO or MORE hotel names
            -> Choose COMPARE_HOTELS (not DESCRIBE_HOTEL)

         4. If the query mentions ONLY ONE specific hotel name
            -> Choose DESCRIBE_HOTEL (not LIST_HOTELS)

         5. If the query is about "visa" or "entry requirements"
            -> Choose CHECK_VISA

         6. If the query uses only neutral search words ("show", "find", "list") WITHOUT opinion keywords
            -> Choose LIST_HOTELS

         7. If the query is COMPLETELY unrelated to hotels or travel
            -> Return NONE

         ═══════════════════════════════════════════════════════════════════════════════

         **User Query:** "{user_query}"

         **Your Response (return ONLY ONE of these):**
         LIST_HOTELS
         RECOMMEND_HOTEL
         DESCRIBE_HOTEL
         COMPARE_HOTELS
         CHECK_VISA
         NONE
         """

print("IntentClassifier class defined")

IntentClassifier class defined


### Testing Intent Classification

We test all 5 intents plus out-of-scope queries to validate the classifier's accuracy.

In [4]:
# Initialize classifier
classifier = IntentClassifier()

# Comprehensive test cases: (query, expected_intent)
test_cases = [
    # ===== LIST_HOTELS - neutral searching =====
    ("Show me hotels in Paris", "LIST_HOTELS"),
    ("Find 5-star hotels in Dubai", "LIST_HOTELS"),
    ("List hotels near the beach in Rome", "LIST_HOTELS"),
    ("Hotels in Cairo", "LIST_HOTELS"),
    ("Search for hotels in Berlin city center", "LIST_HOTELS"),
    
    # ===== RECOMMEND_HOTEL - advice/opinion/personalization =====
    ("Recommend a hotel for families in Dubai", "RECOMMEND_HOTEL"),
    ("What's the best cheap hotel in Paris?", "RECOMMEND_HOTEL"),
    ("I'm 25 traveling solo, suggest a clean hotel in Cairo", "RECOMMEND_HOTEL"),
    ("Top hotels in Rome", "RECOMMEND_HOTEL"),
    ("Which hotel should I book in Berlin?", "RECOMMEND_HOTEL"),
    ("Suggest a romantic hotel for our honeymoon", "RECOMMEND_HOTEL"),
    ("Best hotel for business travelers in London", "RECOMMEND_HOTEL"),
    
    # ===== DESCRIBE_HOTEL - single hotel info =====
    ("Tell me about Hilton Cairo", "DESCRIBE_HOTEL"),
    ("What is The Azure Tower like?", "DESCRIBE_HOTEL"),
    ("Information about Marriott Dubai", "DESCRIBE_HOTEL"),
    ("Describe the Grand Hotel Paris", "DESCRIBE_HOTEL"),
    
    # ===== COMPARE_HOTELS - compare multiple hotels =====
    ("Compare Hilton Cairo and Marriott Cairo", "COMPARE_HOTELS"),
    ("Which is better: The Azure Tower or Nile Grandeur?", "COMPARE_HOTELS"),
    ("Hotel A vs Hotel B in Paris", "COMPARE_HOTELS"),
    ("Difference between Sheraton and Radisson in Dubai", "COMPARE_HOTELS"),
    
    # ===== CHECK_VISA - visa requirements =====
    ("Do Egyptians need a visa for Turkey?", "CHECK_VISA"),
    ("Visa requirements from UK to UAE", "CHECK_VISA"),
    ("Do Indians need a visa to visit Germany?", "CHECK_VISA"),
    ("What are the entry requirements for France from USA?", "CHECK_VISA"),
    
    # ===== Out of scope - should return None =====
    ("What's the weather like today?", None),
    ("Tell me a joke", None),
    ("How do I cook pasta?", None),
    ("What's the capital of France?", None),
]

print("=" * 100)
print("TESTING INTENT CLASSIFICATION")
print("=" * 100)

num_total = len(test_cases)
num_correct = 0
failures = []

for i, (query, expected) in enumerate(test_cases, 1):
    predicted = classifier.classify(query)
    is_correct = (predicted == expected)
    
    if is_correct:
        num_correct += 1
        status = "[PASS]"
    else:
        status = "[FAIL]"
        failures.append((query, expected, predicted))
    
    # Format output
    print(f"\n[{i}/{num_total}] {status}")
    print(f"Query:     {query}")
    print(f"Expected:  {expected}")
    print(f"Predicted: {predicted}")
    
    if predicted is None:
        print(f"System Response: 'I cannot help with this request. I can assist with hotel search, recommendations, visa information, and hotel comparisons.'")

print("\n" + "=" * 100)
print(f"SUMMARY: {num_correct}/{num_total} correct ({(num_correct / num_total) * 100:.1f}% accuracy)")
print("=" * 100)

if failures:
    print(f"\nFAILED CASES ({len(failures)}):")
    for query, expected, predicted in failures:
        print(f"  - '{query}'")
        print(f"    Expected: {expected}, Got: {predicted}")
else:
    print("\nALL TESTS PASSED!")

TESTING INTENT CLASSIFICATION

[1/28] [PASS]
Query:     Show me hotels in Paris
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[2/28] [PASS]
Query:     Find 5-star hotels in Dubai
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[3/28] [PASS]
Query:     List hotels near the beach in Rome
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[4/28] [PASS]
Query:     Hotels in Cairo
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[5/28] [PASS]
Query:     Search for hotels in Berlin city center
Expected:  LIST_HOTELS
Predicted: LIST_HOTELS

[6/28] [PASS]
Query:     Recommend a hotel for families in Dubai
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HOTEL

[7/28] [PASS]
Query:     What's the best cheap hotel in Paris?
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HOTEL

[8/28] [PASS]
Query:     I'm 25 traveling solo, suggest a clean hotel in Cairo
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HOTEL

[9/28] [PASS]
Query:     Top hotels in Rome
Expected:  RECOMMEND_HOTEL
Predicted: RECOMMEND_HO

---
<a id='part1b'></a>
## 1.b Entity Extraction

### Purpose
Extract structured information from user queries to fill Cypher query parameters.

### Design Decision: LLM + Normalization + Schema Enforcement

We use a **three-stage approach**:
1. **LLM Extraction** (OpenAI GPT-4o-mini) - Extract raw entities as JSON
2. **Schema Enforcement** - Validate against intent-specific schemas
3. **Normalization** - Standardize values (star_rating → int, aspects → list, etc.)

**Critical Behavioral Rules:**
- Extract ONLY explicitly mentioned entities
- Vague words like "good", "best", "nice" do **NOT** trigger aspect extraction
- Aspects extracted **ONLY** if specifically mentioned (e.g., "clean rooms" → cleanliness)

In [9]:
import json

# Initialize OpenAI client for entity extraction (reusing same model as intent classifier)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("OpenAI client initialized for entity extraction")

OpenAI client initialized for entity extraction


### Entity Schemas by Intent

Each intent requires different entities:

| Intent | Entities |
|--------|----------|
| **LIST_HOTELS** | city, country, star_rating |
| **RECOMMEND_HOTEL** | city, country, traveller_type, age_group, user_gender, star_rating, aspects |
| **DESCRIBE_HOTEL** | hotel_name, aspects |
| **COMPARE_HOTELS** | hotel1, hotel2, traveller_type, aspects |
| **CHECK_VISA** | from_country, to_country |

In [10]:
# ============================================================================
# Intent-Aware Schemas
# ============================================================================

SCHEMAS: Dict[str, Dict[str, Any]] = {
    "LIST_HOTELS": {
        "city": None,
        "country": None,
        "star_rating": None
    },
    "RECOMMEND_HOTEL": {
        "city": None,
        "country": None,
        "traveller_type": None,
        "age_group": None,
        "user_gender": None,
        "star_rating": None,
        "aspects": None
    },
    "DESCRIBE_HOTEL": {
        "hotel_name": None,
        "aspects": None
    },
    "COMPARE_HOTELS": {
        "hotel1": None,
        "hotel2": None,
        "traveller_type": None,
        "aspects": None
    },
    "CHECK_VISA": {
        "from_country": None,
        "to_country": None
    }
}

# Allowed aspects from dataset
ALLOWED_ASPECTS = [
    "cleanliness", "comfort", "facilities",
    "location", "staff", "value_for_money"
]

print("Schemas defined successfully")
print(f"Intents: {list(SCHEMAS.keys())}")
print(f"Allowed aspects: {ALLOWED_ASPECTS}")

Schemas defined successfully
Intents: ['LIST_HOTELS', 'RECOMMEND_HOTEL', 'DESCRIBE_HOTEL', 'COMPARE_HOTELS', 'CHECK_VISA']
Allowed aspects: ['cleanliness', 'comfort', 'facilities', 'location', 'staff', 'value_for_money']


In [11]:
def enforce_schema(intent: str, entities: Dict[str, Any]) -> Dict[str, Any]:
    """
    Enforce schema constraints and normalize values.
    
    Rules:
    - Drop keys not in SCHEMAS[intent]
    - Ensure all schema keys exist (missing → None)
    - Normalize star_rating → int (1-5)
    - Normalize aspects → list[str], lowercase, deduplicated
    - Normalize traveller_type, user_gender to valid values
    """
    schema = SCHEMAS[intent]
    result = {}
    
    for key in schema.keys():
        value = entities.get(key, None)
        
        # Normalize star_rating → int
        if key == "star_rating" and value is not None:
            try:
                value = int(value)
                if not (1 <= value <= 5):
                    value = None
            except (ValueError, TypeError):
                value = None
        
        # Normalize aspects → list[str]
        elif key == "aspects" and value is not None:
            if isinstance(value, str):
                value = [value]
            if isinstance(value, list):
                # Lowercase, snake_case, deduplicate, filter allowed
                normalized = []
                for asp in value:
                    if isinstance(asp, str):
                        asp_clean = asp.lower().strip().replace(" ", "_").replace("-", "_")
                        if asp_clean in ALLOWED_ASPECTS:
                            normalized.append(asp_clean)
                value = list(set(normalized)) if normalized else None
            else:
                value = None
        
        # Normalize traveller_type
        elif key == "traveller_type" and value is not None:
            if isinstance(value, str):
                value = value.lower().strip()
                if value not in ["family", "solo", "couple", "business", "group"]:
                    value = None
        
        # Normalize user_gender
        elif key == "user_gender" and value is not None:
            if isinstance(value, str):
                value = value.lower().strip()
                if value not in ["male", "female"]:
                    value = None
        
        result[key] = value
    
    return result

print("enforce_schema() function defined")

enforce_schema() function defined


In [12]:
def extract_entities(text: str, intent: str) -> Dict[str, Any]:
    """
    Extract entities from user query using GPT-4o-mini (same model as intent classifier).
    
    Args:
        text: User query
        intent: One of the 5 intents (LIST_HOTELS, RECOMMEND_HOTEL, etc.)
    
    Returns:
        Schema-compliant dictionary with extracted entities
    """
    if intent not in SCHEMAS:
        raise ValueError(f"Unknown intent: {intent}")
    
    # Build extraction prompt
    prompt = f"""Extract entities from this user query and return ONLY valid JSON.

Query: "{text}"
Intent: {intent}
Required keys: {list(SCHEMAS[intent].keys())}

CRITICAL RULES:
1. Extract ONLY explicitly mentioned entities
2. Missing information → null
3. Do NOT interpret "good", "best", "nice", "top" as entities
4. For aspects: ONLY extract if explicitly mentioned
   - "clean" → cleanliness
   - "comfortable" → comfort
   - "good location" → location
   - "staff" → staff
   - "facilities" → facilities
   - "cheap"/"value" → value_for_money
   
5. Do NOT extract aspects from vague words:
   - "good", "best", "nice", "top", "quality" → extract NO aspects

6. Allowed aspects: {ALLOWED_ASPECTS}
7. traveller_type: family, solo, couple, business, group
8. user_gender: male, female
9. star_rating: 1-5 (numeric only)

Return ONLY JSON matching this schema: {SCHEMAS[intent]}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Same model as intent classifier
            messages=[
                {"role": "system", "content": "You are a precise NER system. Return ONLY valid JSON, no markdown, no explanations."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=200
        )
        
        raw = response.choices[0].message.content.strip()
        
        # Strip markdown code blocks if present
        if raw.startswith("```"):
            raw = raw.split("```")[1]
            if raw.startswith("json"):
                raw = raw[4:]
            raw = raw.strip()
        
        entities = json.loads(raw)
        
        # Enforce schema constraints
        return enforce_schema(intent, entities)
        
    except Exception as e:
        print(f"Error extracting entities: {e}")
        # Return empty schema on error
        return dict(SCHEMAS[intent])

print("extract_entities() function defined")

extract_entities() function defined


### Testing Entity Extraction

We test the NER system with three critical examples:
1. **Explicit aspects** - Aspects clearly mentioned should be extracted
2. **Vague quality words** - "good", "best" should NOT trigger aspect extraction
3. **Comparison query** - Extract multiple hotels and specific aspects

In [15]:
print("=" * 100)
print("TESTING ENTITY EXTRACTION")
print("=" * 100)

# ============================================================================
# Example 1: Explicit aspects mentioned
# ============================================================================
query1 = "Recommend a hotel for families in Dubai with good location and clean rooms"
intent1 = "RECOMMEND_HOTEL"
result1 = extract_entities(query1, intent1)

print("\n[Example 1] Explicit Aspects")
print(f"Query:  {query1}")
print(f"Intent: {intent1}")
print(f"Output: {json.dumps(result1, indent=2)}")
print("Expected: traveller_type='family', city='Dubai', aspects=['location', 'cleanliness']")
print("Note: 'good location' → location aspect, 'clean rooms' → cleanliness aspect")

# ============================================================================
# Example 2: Vague quality words (NO aspects should be extracted)
# ============================================================================
query2 = "I want good hotels for family in Paris"
intent2 = "RECOMMEND_HOTEL"
result2 = extract_entities(query2, intent2)

print("\n[Example 2] Vague Quality Words (Critical Test)")
print(f"Query:  {query2}")
print(f"Intent: {intent2}")
print(f"Output: {json.dumps(result2, indent=2)}")
print("Expected: traveller_type='family', city='Paris', aspects=None")
print("Note: 'good' is vague and should NOT extract any aspect")

# ============================================================================
# Example 3: Compare hotels with explicit aspects
# ============================================================================
query3 = "Compare Hilton Cairo and Marriott Cairo for staff quality and comfort"
intent3 = "COMPARE_HOTELS"
result3 = extract_entities(query3, intent3)

print("\n[Example 3] Comparison with Explicit Aspects")
print(f"Query:  {query3}")
print(f"Intent: {intent3}")
print(f"Output: {json.dumps(result3, indent=2)}")
print("Expected: hotel1='Hilton Cairo', hotel2='Marriott Cairo', aspects=['staff', 'comfort']")
print("Note: 'staff quality' → staff aspect, 'comfort' → comfort aspect")

print("\n" + "=" * 100)
print("ENTITY EXTRACTION TESTING COMPLETE")
print("=" * 100)
print("=" * 100)
print("EXTENDED ENTITY EXTRACTION STRESS TEST")
print("=" * 100)

# =====================================================================
# Example 4: Multiple aspects + traveller type + star rating
# =====================================================================
query4 = "Recommend a 5 star hotel for business travelers in London with good staff and comfort"
intent4 = "RECOMMEND_HOTEL"
result4 = extract_entities(query4, intent4)

print("\n[Example 4] Multiple Explicit Aspects + Star Rating")
print(f"Query:  {query4}")
print(f"Intent: {intent4}")
print(f"Output: {json.dumps(result4, indent=2)}")
print("Expected: city='London', star_rating=5, traveller_type='business', aspects=['staff','comfort']")
print("Note: 'good staff' → staff, 'comfort' → comfort")

# =====================================================================
# Example 5: No aspects, no city (pure traveller preference)
# =====================================================================
query5 = "I want hotels for solo travelers"
intent5 = "RECOMMEND_HOTEL"
result5 = extract_entities(query5, intent5)

print("\n[Example 5] Traveller Type Only")
print(f"Query:  {query5}")
print(f"Intent: {intent5}")
print(f"Output: {json.dumps(result5, indent=2)}")
print("Expected: traveller_type='solo', all other fields=None")

# =====================================================================
# Example 6: LIST_HOTELS should ignore traveller types and aspects
# =====================================================================
query6 = "List hotels in Rome for families with clean rooms"
intent6 = "LIST_HOTELS"
result6 = extract_entities(query6, intent6)

print("\n[Example 6] LIST_HOTELS Ignores Aspects & Traveller Type")
print(f"Query:  {query6}")
print(f"Intent: {intent6}")
print(f"Output: {json.dumps(result6, indent=2)}")
print("Expected: city='Rome', aspects ignored, traveller_type ignored")

# =====================================================================
# Example 7: DESCRIBE_HOTEL without aspects
# =====================================================================
query7 = "Describe Hilton Dubai"
intent7 = "DESCRIBE_HOTEL"
result7 = extract_entities(query7, intent7)

print("\n[Example 7] Describe Hotel (No Aspects)")
print(f"Query:  {query7}")
print(f"Intent: {intent7}")
print(f"Output: {json.dumps(result7, indent=2)}")
print("Expected: hotel_name='Hilton Dubai', aspects=None")

# =====================================================================
# Example 8: DESCRIBE_HOTEL with aspects
# =====================================================================
query8 = "Describe Hilton Dubai focusing on location and facilities"
intent8 = "DESCRIBE_HOTEL"
result8 = extract_entities(query8, intent8)

print("\n[Example 8] Describe Hotel with Explicit Aspects")
print(f"Query:  {query8}")
print(f"Intent: {intent8}")
print(f"Output: {json.dumps(result8, indent=2)}")
print("Expected: hotel_name='Hilton Dubai', aspects=['location','facilities']")

# =====================================================================
# Example 9: CHECK_VISA intent
# =====================================================================
query9 = "Do Egyptians need a visa for Turkey?"
intent9 = "CHECK_VISA"
result9 = extract_entities(query9, intent9)

print("\n[Example 9] Visa Query")
print(f"Query:  {query9}")
print(f"Intent: {intent9}")
print(f"Output: {json.dumps(result9, indent=2)}")
print("Expected: from_country='Egypt', to_country='Turkey'")

# =====================================================================
# Example 10: Noise words + vague adjectives
# =====================================================================
query10 = "I want really amazing and top hotels for families in Madrid"
intent10 = "RECOMMEND_HOTEL"
result10 = extract_entities(query10, intent10)

print("\n[Example 10] Noise & Vague Adjectives")
print(f"Query:  {query10}")
print(f"Intent: {intent10}")
print(f"Output: {json.dumps(result10, indent=2)}")
print("Expected: city='Madrid', traveller_type='family', aspects=None")
print("Note: 'amazing', 'top' must be ignored")

# =====================================================================
# Example 11: Multiple cities (edge case)
# =====================================================================
query11 = "Compare hotels in Paris and London for location"
intent11 = "COMPARE_HOTELS"
result11 = extract_entities(query11, intent11)


print("\n" + "=" * 100)
print("EXTENDED ENTITY EXTRACTION TESTING COMPLETE")
print("=" * 100)


TESTING ENTITY EXTRACTION

[Example 1] Explicit Aspects
Query:  Recommend a hotel for families in Dubai with good location and clean rooms
Intent: RECOMMEND_HOTEL
Output: {
  "city": "Dubai",
  "country": null,
  "traveller_type": "family",
  "age_group": null,
  "user_gender": null,
  "star_rating": null,
  "aspects": [
    "location",
    "cleanliness"
  ]
}
Expected: traveller_type='family', city='Dubai', aspects=['location', 'cleanliness']
Note: 'good location' → location aspect, 'clean rooms' → cleanliness aspect

[Example 2] Vague Quality Words (Critical Test)
Query:  I want good hotels for family in Paris
Intent: RECOMMEND_HOTEL
Output: {
  "city": "Paris",
  "country": null,
  "traveller_type": "family",
  "age_group": null,
  "user_gender": null,
  "star_rating": null,
  "aspects": null
}
Expected: traveller_type='family', city='Paris', aspects=None
Note: 'good' is vague and should NOT extract any aspect

[Example 3] Comparison with Explicit Aspects
Query:  Compare Hilton Cair